<a href="https://colab.research.google.com/github/jass-jones/Deep_Learning_challenge/blob/main/Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv('/content/drive/MyDrive/charity_data.csv')
application_df.head()

Mounted at /content/drive


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop('EIN', axis= 1)
application_df = application_df.drop('NAME', axis= 1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
counts = application_df.nunique()
print(counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
application_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


## Optimizing the model:

In [13]:
application_types_to_replace = list(application_counts[application_counts < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [14]:
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2170        1
C4200        1
C1370        1
C1728        1
C1820        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [15]:
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [16]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(application_df, drop_first= True)
dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,108590,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1,6692,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0
4,1,142590,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0


In [18]:
y = dummies.IS_SUCCESSFUL.values
X = dummies.drop('IS_SUCCESSFUL', axis= 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify= y)

In [19]:
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
features = len(X_train_scaled[0])
node = 80
sec_node = 30
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units= node, input_dim= features, activation= 'relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units= sec_node, activation= 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units= 1, activation= 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2720      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,181
Trainable params: 5,181
Non-trainable params: 0
_________________________________________________________________


In [21]:
nn.compile(loss= "binary_crossentropy", optimizer= "adam", metrics= ["accuracy"])

In [22]:
fit_model = nn.fit(X_train_scaled, y_train, epochs= 100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5742 - accuracy: 0.7191
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5589 - accuracy: 0.7282
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5558 - accuracy: 0.7320
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5540 - accuracy: 0.7334
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5526 - accuracy: 0.7334
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5520 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7341
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7337
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7339
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accura

In [23]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5645 - accuracy: 0.7236 - 456ms/epoch - 2ms/step
Loss: 0.5644608736038208, Accuracy: 0.7236151695251465


## Adding additinal Epochs

In [29]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=250)

Epoch 1/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5388 - accuracy: 0.7394
Epoch 2/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5389 - accuracy: 0.7406
Epoch 3/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5386 - accuracy: 0.7400
Epoch 4/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5387 - accuracy: 0.7407
Epoch 5/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5384 - accuracy: 0.7397
Epoch 6/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5385 - accuracy: 0.7397
Epoch 7/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5382 - accuracy: 0.7403
Epoch 8/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5385 - accuracy: 0.7402
Epoch 9/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5384 - accuracy: 0.7403
Epoch 10/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5381 - accura

In [30]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5957 - accuracy: 0.7249 - 320ms/epoch - 1ms/step
Loss: 0.5957299470901489, Accuracy: 0.7248979806900024


## Optimization appears to not have a better outcome for our model

In [31]:
nn.save("AlphabetSoup_Optimized")

INFO:tensorflow:Assets written to: AlphabetSoup_Optimized/assets
